In [1]:
# %% [markdown]
# # CycleGAN Implementation (PyTorch)
# Unpaired Image-to-Image Translation

# %%
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# %% [markdown]
# ## 1. Model Components

# %%
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, 3, 1, 1),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, 3, 1, 1),
            nn.InstanceNorm2d(channels)
        )

    def forward(self, x):
        return x + self.block(x)

# %%
def generator_block(in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
        nn.InstanceNorm2d(out_channels),
        nn.ReLU(inplace=True)
    )

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            # Encoding
            nn.ReflectionPad2d(3),
            nn.Conv2d(3, 64, 7, 1, 0),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
            
            generator_block(64, 128, 3, 2, 1),
            generator_block(128, 256, 3, 2, 1),
            
            # Residual blocks
            *[ResidualBlock(256) for _ in range(9)],
            
            # Decoding
            nn.ConvTranspose2d(256, 128, 3, 2, 1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),
            
            nn.ConvTranspose2d(128, 64, 3, 2, 1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.ReflectionPad2d(3),
            nn.Conv2d(64, 3, 7, 1, 0),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

# %%
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        def discriminator_block(in_filters, out_filters, normalize=True):
            layers = [nn.Conv2d(in_filters, out_filters, 4, 2, 1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(3, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.ZeroPad2d((1, 0, 1, 0)),
            nn.Conv2d(512, 1, 4, padding=1)
        )

    def forward(self, img):
        return self.model(img)

# %% [markdown]
# ## 2. Training Components

# %%
class CycleGAN:
    def __init__(self, lr=0.0002, lambda_cycle=10.0):
        # Initialize generators and discriminators
        self.G_AB = Generator()  # A -> B
        self.G_BA = Generator()  # B -> A
        self.D_A = Discriminator()  # Discriminator for domain A
        self.D_B = Discriminator()  # Discriminator for domain B
        
        # Loss functions
        self.criterion_GAN = nn.MSELoss()
        self.criterion_cycle = nn.L1Loss()
        self.lambda_cycle = lambda_cycle
        
        # Optimizers
        self.optimizer_G = optim.Adam(
            list(self.G_AB.parameters()) + list(self.G_BA.parameters()),
            lr=lr, betas=(0.5, 0.999)
        )
        self.optimizer_D_A = optim.Adam(self.D_A.parameters(), lr=lr, betas=(0.5, 0.999))
        self.optimizer_D_B = optim.Adam(self.D_B.parameters(), lr=lr, betas=(0.5, 0.999))

# %% [markdown]
# ## 3. Visualization Functions

# %%
def plot_images(real_A, fake_B, reconstructed_A):
    """Visualize the translation cycle"""
    images = [real_A, fake_B, reconstructed_A]
    titles = ["Real Domain A", "Generated Domain B", "Reconstructed Domain A"]
    
    plt.figure(figsize=(12, 4))
    for i, (img, title) in enumerate(zip(images, titles)):
        img = img.detach().cpu().squeeze(0)
        img = img.permute(1, 2, 0).numpy()
        img = 0.5 * img + 0.5  # Denormalize
        
        plt.subplot(1, 3, i + 1)
        plt.imshow(np.clip(img, 0, 1))
        plt.title(title)
        plt.axis("off")
    
    plt.tight_layout()
    plt.show()

def save_images(real_A, fake_B, reconstructed_A, epoch, batch_idx):
    """Save images to file"""
    transform = transforms.ToPILImage()
    
    def prepare_image(tensor):
        img = tensor.detach().cpu().squeeze(0)
        img = 0.5 * img + 0.5  # Denormalize
        return transform(img)
    
    prepare_image(real_A).save(f"real_A_epoch{epoch}_batch{batch_idx}.png")
    prepare_image(fake_B).save(f"fake_B_epoch{epoch}_batch{batch_idx}.png")
    prepare_image(reconstructed_A).save(f"reconstructed_A_epoch{epoch}_batch{batch_idx}.png")

# %% [markdown]
# ## 4. Training Loop (Simplified)

# %%
def train_step(cyclegan, real_A, real_B):
    """Single training step"""
    
    # Train Generators
    cyclegan.optimizer_G.zero_grad()
    
    # Generate fake images
    fake_B = cyclegan.G_AB(real_A)
    fake_A = cyclegan.G_BA(real_B)
    
    # GAN losses
    loss_GAN_AB = cyclegan.criterion_GAN(cyclegan.D_B(fake_B), torch.ones_like(cyclegan.D_B(fake_B)))
    loss_GAN_BA = cyclegan.criterion_GAN(cyclegan.D_A(fake_A), torch.ones_like(cyclegan.D_A(fake_A)))
    
    # Cycle consistency losses
    reconstructed_A = cyclegan.G_BA(fake_B)
    loss_cycle_A = cyclegan.criterion_cycle(reconstructed_A, real_A)
    
    reconstructed_B = cyclegan.G_AB(fake_A)
    loss_cycle_B = cyclegan.criterion_cycle(reconstructed_B, real_B)
    
    # Total generator loss
    loss_G = loss_GAN_AB + loss_GAN_BA + \
             cyclegan.lambda_cycle * (loss_cycle_A + loss_cycle_B)
    
    loss_G.backward()
    cyclegan.optimizer_G.step()
    
    return fake_B, fake_A, reconstructed_A, reconstructed_B, loss_G.item()

# %% [markdown]
# ## 5. Example Usage

# %%
def main():
    # Initialize CycleGAN
    cyclegan = CycleGAN()
    
    # Create dummy data (replace with real dataset)
    batch_size = 1
    real_A = torch.randn(batch_size, 3, 256, 256)
    real_B = torch.randn(batch_size, 3, 256, 256)
    
    # Single training step
    print("Training one step...")
    fake_B, fake_A, reconstructed_A, reconstructed_B, loss = train_step(cyclegan, real_A, real_B)
    
    print(f"Generator Loss: {loss:.4f}")
    
    # Visualize results
    plot_images(real_A, fake_B, reconstructed_A)
    
    # Save images
    save_images(real_A, fake_B, reconstructed_A, epoch=0, batch_idx=0)

# %%
if __name__ == "__main__":
    main()

OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "c:\users\user\AppData\Roaming\Python\Python312\site-packages\torch\lib\c10.dll" or one of its dependencies.